In [3]:
import sys
sys.path.append("..")
sys.path.append("../models")

from models import ResNetSimCLR
import torch 

model = ResNetSimCLR(size=50, device="cuda", embedding_dim=64)
checkpoint = torch.load("../checkpoints/checkpoint_epoch_80.pt", map_location="cuda")

model.load_state_dict(checkpoint["model_state_dict"])

# Freeze all parameters in ResNetSimCLR
for param in model.parameters():
    param.requires_grad = False

GPU is being utilized


/tmp/ipykernel_3002/1677408530.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("../checkpoints/checkpoint_epoch_80.pt", map_location="cuda")


In [4]:
import torch
import torch.nn as nn

class ClassifierHead(nn.Module):
    def __init__(self, base_model):
        super(ClassifierHead, self).__init__()
        self.encoder = base_model  # Use the existing model as the encoder
        # Add new layers on top of the encoder
        self.fc1 = nn.Linear(base_model.embedding_dim, 64)  # First FC layer
        self.relu = nn.ReLU()  # Activation function
        self.fc2 = nn.Linear(64, 10)  # Classification head for 10 classes
        self.softmax = nn.Softmax(dim=1)  # Softmax for probabilities
    
    def forward(self, x):
        # Forward pass through the encoder
        x = self.encoder(x)
        # Pass through the new fully connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)  # Apply softmax for probabilities
        return x

# Instantiate the custom model
custom_model = ClassifierHead(model)

# Verify trainable parameters
for name, param in custom_model.named_parameters():
    print(f"{name}: {'Trainable' if param.requires_grad else 'Frozen'}")

encoder.model.conv1.weight: Frozen
encoder.model.bn1.weight: Frozen
encoder.model.bn1.bias: Frozen
encoder.model.layer1.0.conv1.weight: Frozen
encoder.model.layer1.0.bn1.weight: Frozen
encoder.model.layer1.0.bn1.bias: Frozen
encoder.model.layer1.0.conv2.weight: Frozen
encoder.model.layer1.0.bn2.weight: Frozen
encoder.model.layer1.0.bn2.bias: Frozen
encoder.model.layer1.0.conv3.weight: Frozen
encoder.model.layer1.0.bn3.weight: Frozen
encoder.model.layer1.0.bn3.bias: Frozen
encoder.model.layer1.0.downsample.0.weight: Frozen
encoder.model.layer1.0.downsample.1.weight: Frozen
encoder.model.layer1.0.downsample.1.bias: Frozen
encoder.model.layer1.1.conv1.weight: Frozen
encoder.model.layer1.1.bn1.weight: Frozen
encoder.model.layer1.1.bn1.bias: Frozen
encoder.model.layer1.1.conv2.weight: Frozen
encoder.model.layer1.1.bn2.weight: Frozen
encoder.model.layer1.1.bn2.bias: Frozen
encoder.model.layer1.1.conv3.weight: Frozen
encoder.model.layer1.1.bn3.weight: Frozen
encoder.model.layer1.1.bn3.bias: F

In [5]:
criterion = nn.CrossEntropyLoss()  # Suitable for logits or probabilities
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, custom_model.parameters()), lr=0.001)

In [6]:
print("Parameters being optimized:")
for i, param_group in enumerate(optimizer.param_groups):
    print(f"Parameter group {i}:")
    for param in param_group["params"]:
        print(param.shape)

Parameters being optimized:
Parameter group 0:
torch.Size([64, 64])
torch.Size([64])
torch.Size([10, 64])
torch.Size([10])


In [7]:
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

data_dir="../dataset/data"
mean=[0.4914, 0.4822, 0.4465]
std=[0.2470, 0.2435, 0.2616]
img_size = 32
batch_size = 32

transform = transforms.Compose([
                transforms.Resize(img_size),
                transforms.ToTensor(),
                transforms.Normalize(mean=mean, std=std)
            ])

dataset_train = CIFAR10(root=data_dir, train=True, download=True, transform=transform)
subset_indices_train = list(range(1025, 2025))
small_dataset_train = torch.utils.data.Subset(dataset_train, subset_indices_train)
data_loader_train = DataLoader(small_dataset_train, batch_size=batch_size, shuffle=True)

dataset_test = CIFAR10(root=data_dir, train=False, download=True, transform=transform)
subset_indices_test = list(range(257, 457))
small_dataset_test = torch.utils.data.Subset(dataset_test, subset_indices_test)
data_loader_test = DataLoader(small_dataset_test, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Hyperparameters
num_epochs = 20
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, custom_model.parameters()), lr=learning_rate)

# Move model to device
custom_model.to(device)

# Training loop with validation
for epoch in range(num_epochs):
    # Training phase
    custom_model.train()
    train_loss = 0
    correct_train = 0
    total_train = 0

    for batch_idx, (inputs, targets) in enumerate(tqdm(data_loader_train, desc=f"Epoch {epoch+1}/{num_epochs} - Training")):
        inputs, targets = inputs.to(device), targets.to(device)

        # Zero the gradient buffers
        optimizer.zero_grad()

        # Forward pass
        outputs = custom_model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track metrics
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total_train += targets.size(0)
        correct_train += predicted.eq(targets).sum().item()

    train_acc = 100. * correct_train / total_train
    avg_train_loss = train_loss / len(data_loader_train)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}%")

    # Validation phase
    custom_model.eval()
    val_loss = 0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(data_loader_test, desc=f"Epoch {epoch+1}/{num_epochs} - Validation")):
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = custom_model(inputs)
            loss = criterion(outputs, targets)

            # Track metrics
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total_val += targets.size(0)
            correct_val += predicted.eq(targets).sum().item()

    val_acc = 100. * correct_val / total_val
    avg_val_loss = val_loss / len(data_loader_test)

    print(f"Epoch {epoch+1}/{num_epochs} - Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

Epoch 1/20 - Training:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 1/20 - Training: 100%|██████████| 32/32 [00:00<00:00, 68.30it/s]


Epoch 1/20 - Train Loss: 2.2983 | Train Acc: 15.40%


Epoch 1/20 - Validation: 100%|██████████| 7/7 [00:00<00:00, 166.57it/s]


Epoch 1/20 - Val Loss: 2.2906 | Val Acc: 20.00%


...

Epoch 19/20 - Val Loss: 2.1739 | Val Acc: 25.50%


Epoch 20/20 - Training: 100%|██████████| 32/32 [00:00<00:00, 129.72it/s]


Epoch 20/20 - Train Loss: 2.1456 | Train Acc: 31.60%


Epoch 20/20 - Validation: 100%|██████████| 7/7 [00:00<00:00, 170.69it/s]

Epoch 20/20 - Val Loss: 2.1748 | Val Acc: 25.00%


In [9]:
import torch
import torch.nn as nn
from torchvision.models import resnet50

# Define the ClassifierHead
class ClassifierHead(nn.Module):
    def __init__(self, base_model, embedding_dim):
        super(ClassifierHead, self).__init__()
        self.encoder = base_model  # Use the base model as the encoder
        self.fc1 = nn.Linear(embedding_dim, 64)  # First FC layer
        self.relu = nn.ReLU()  # Activation function
        self.fc2 = nn.Linear(64, 10)  # Classification head for 10 classes
        self.softmax = nn.Softmax(dim=1)  # Softmax for probabilities
    
    def forward(self, x):
        # Forward pass through the encoder
        x = self.encoder(x)
        # Pass through the new fully connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)  # Apply softmax for probabilities
        return x

# Initialize a randomly initialized ResNet
random_resnet = resnet50(pretrained=False)

# Replace the final fully connected layer with an identity function
random_resnet.fc = nn.Identity()

# Freeze all parameters in the ResNet
for param in random_resnet.parameters():
    param.requires_grad = False

# Define the embedding dimension (ResNet-50's output features are 2048)
embedding_dim = 2048

# Instantiate the custom model
custom_model = ClassifierHead(random_resnet, embedding_dim)

# Verify trainable parameters
print("Trainable Parameters:")
for name, param in custom_model.named_parameters():
    print(f"{name}: {'Trainable' if param.requires_grad else 'Frozen'}")


/mnt/new_storage2/Revisiting-Reverse-Distillation/RRDV/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/new_storage2/Revisiting-Reverse-Distillation/RRDV/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Trainable Parameters:
encoder.conv1.weight: Frozen
encoder.bn1.weight: Frozen
encoder.bn1.bias: Frozen
encoder.layer1.0.conv1.weight: Frozen
encoder.layer1.0.bn1.weight: Frozen
encoder.layer1.0.bn1.bias: Frozen
encoder.layer1.0.conv2.weight: Frozen
encoder.layer1.0.bn2.weight: Frozen
encoder.layer1.0.bn2.bias: Frozen
encoder.layer1.0.conv3.weight: Frozen
encoder.layer1.0.bn3.weight: Frozen
encoder.layer1.0.bn3.bias: Frozen
encoder.layer1.0.downsample.0.weight: Frozen
encoder.layer1.0.downsample.1.weight: Frozen
encoder.layer1.0.downsample.1.bias: Frozen
encoder.layer1.1.conv1.weight: Frozen
encoder.layer1.1.bn1.weight: Frozen
encoder.layer1.1.bn1.bias: Frozen
encoder.layer1.1.conv2.weight: Frozen
encoder.layer1.1.bn2.weight: Frozen
encoder.layer1.1.bn2.bias: Frozen
encoder.layer1.1.conv3.weight: Frozen
encoder.layer1.1.bn3.weight: Frozen
encoder.layer1.1.bn3.bias: Frozen
encoder.layer1.2.conv1.weight: Frozen
encoder.layer1.2.bn1.weight: Frozen
encoder.layer1.2.bn1.bias: Frozen
encoder.

In [12]:
import torch
import torch.nn as nn
from torchvision.models import resnet50
from tqdm import tqdm

random_resnet = resnet50(pretrained=False)
random_resnet.fc = nn.Identity()  # Replace the final layer with identity
for param in random_resnet.parameters():
    param.requires_grad = False  # Freeze all layers

embedding_dim = 2048
custom_model = ClassifierHead(random_resnet, embedding_dim)

# Move the model to GPU
custom_model = custom_model.cuda()

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, custom_model.parameters()), lr=1e-3)

epochs = 20  # Number of epochs
for epoch in range(epochs):
    custom_model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Wrap the batch iteration with tqdm for progress bar
    with tqdm(data_loader_train, desc=f"Epoch {epoch+1}/{epochs} - Training") as pbar_train:
        for inputs, labels in pbar_train:
            inputs, labels = inputs.cuda(), labels.cuda()  # Move inputs and labels to GPU

            optimizer.zero_grad()  # Clear gradients from previous step

            # Forward pass
            outputs = custom_model(inputs)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            loss.backward()  # Backpropagate
            
            optimizer.step()  # Update weights

            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update progress bar
            pbar_train.set_postfix(loss=running_loss / (pbar_train.n + 1), accuracy=100 * correct / total)

    avg_loss = running_loss / len(data_loader_train)
    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
    # Validation loop with tqdm
    custom_model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient computation for validation
        with tqdm(data_loader_test, desc=f"Epoch {epoch+1}/{epochs} - Validation") as pbar_val:
            for inputs, labels in pbar_val:
                inputs, labels = inputs.cuda(), labels.cuda()  # Move inputs and labels to GPU
                
                outputs = custom_model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                # Update progress bar
                pbar_val.set_postfix(accuracy=100 * correct / total)

    val_accuracy = 100 * correct / total
    print(f"Validation Accuracy: {val_accuracy:.2f}%")

Epoch 1/20 - Training: 100%|██████████| 32/32 [00:00<00:00, 120.46it/s, accuracy=10.9, loss=2.85]


Epoch 1/20, Loss: 2.3169, Accuracy: 10.90%


Epoch 1/20 - Validation: 100%|██████████| 7/7 [00:00<00:00, 156.04it/s, accuracy=7.5]


Validation Accuracy: 7.50%


Epoch 2/20 - Training: 100%|██████████| 32/32 [00:00<00:00, 120.83it/s, accuracy=9.3, loss=2.73] 


...

Validation Accuracy: 7.50%


Epoch 20/20 - Training: 100%|██████████| 32/32 [00:00<00:00, 120.60it/s, accuracy=20.7, loss=2.87]


Epoch 20/20, Loss: 2.2392, Accuracy: 20.70%


Epoch 20/20 - Validation: 100%|██████████| 7/7 [00:00<00:00, 157.49it/s, accuracy=14]

Validation Accuracy: 14.00%
